# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform, euclidean
import numpy as np

In [2]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [3]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [4]:
step1=data.groupby(['CustomerID','ProductName'])['Quantity'].sum()
step1=pd.DataFrame(step1)
step1

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [5]:
table = pd.pivot_table(data, values='Quantity', index=['ProductName'],columns=['CustomerID'])
table=table.fillna(0)
table

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yeast Dry - Fermipan,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0
Yoghurt Tubes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0,0.0
"Yogurt - Blueberry, 175 Gr",0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,0.0,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [6]:
distances = pdist(table.T, 'euclidean')
print(distances)
print("-----Squareform-----")
print(squareform(distances))

[ 10.72380529   9.64365076   9.89949494 ... 266.9269563  266.9269563
 269.25824036]
-----Squareform-----
[[  0.          10.72380529   9.64365076 ... 197.94948851 218.8241303
  203.79892051]
 [ 10.72380529   0.          10.67707825 ... 197.34994299 218.62525014
  203.95342606]
 [  9.64365076  10.67707825   0.         ... 197.32460566 218.02981447
  203.80628057]
 ...
 [197.94948851 197.34994299 197.32460566 ...   0.         266.9269563
  266.9269563 ]
 [218.8241303  218.62525014 218.02981447 ... 266.9269563    0.
  269.25824036]
 [203.79892051 203.95342606 203.80628057 ... 266.9269563  269.25824036
    0.        ]]


In [7]:
distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'euclidean'))), index=table.columns, columns=table.columns)
distances

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.085297,0.093953,0.091747,0.087410,0.089695,0.085297,0.088913,0.088152,0.089695,...,0.004809,0.005108,0.004996,0.005421,0.004920,0.005023,0.004880,0.005026,0.004549,0.004883
200,0.085297,1.000000,0.085638,0.085297,0.080070,0.083020,0.084959,0.083651,0.085638,0.087047,...,0.004825,0.005121,0.005014,0.005448,0.004925,0.005032,0.004909,0.005042,0.004553,0.004879
264,0.093953,0.085638,1.000000,0.088152,0.089301,0.087047,0.085638,0.086333,0.087047,0.087047,...,0.004822,0.005115,0.004996,0.005441,0.004932,0.005055,0.004894,0.005042,0.004566,0.004883
356,0.091747,0.085297,0.088152,1.000000,0.085983,0.086688,0.085983,0.091325,0.085983,0.087410,...,0.004814,0.005111,0.004999,0.005437,0.004920,0.005036,0.004871,0.005042,0.004563,0.004886
412,0.087410,0.080070,0.089301,0.085983,1.000000,0.085638,0.085638,0.089301,0.084959,0.087779,...,0.004808,0.005131,0.004996,0.005441,0.004925,0.005042,0.004876,0.005039,0.004568,0.004903
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,0.005023,0.005032,0.005055,0.005036,0.005042,0.005042,0.005042,0.005039,0.005039,0.005033,...,0.003732,0.003945,0.003749,0.004045,0.003964,1.000000,0.003945,0.003945,0.003579,0.003608
98069,0.004880,0.004909,0.004894,0.004871,0.004876,0.004877,0.004882,0.004880,0.004886,0.004886,...,0.003638,0.003765,0.003716,0.003782,0.003888,0.003945,1.000000,0.003907,0.003608,0.003765
98159,0.005026,0.005042,0.005042,0.005042,0.005039,0.005039,0.005039,0.005033,0.005045,0.005036,...,0.003732,0.003984,0.003852,0.003888,0.003817,0.003945,0.003907,1.000000,0.003732,0.003732


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [8]:
#He cogido el Customer 33 para ver cuáles eran los más similares.
similarities = distances[33].sort_values(ascending=False)[1:6]
similarities.keys()

Int64Index([3909, 264, 3531, 2503, 3305], dtype='int64', name='CustomerID')

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [9]:
step5=step1.loc[similarities.keys()]
step5

Quantity
CustomerID ProductName                             
264        Apricots - Halves                      1
           Apricots Fresh                         1
           Bacardi Breezer - Tropical             1
           Bagel - Plain                          1
           Banana - Leaves                        1
...                                             ...
3909       Wine - Charddonnay Errazuriz           2
           Wine - Fume Blanc Fetzer               1
           Wine - Magnotta, Merlot Sr Vqa         1
           Wine - Toasted Head                    1
           Wine - Valpolicella Masi               1

[282 rows x 1 columns]

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [10]:
step5=step5.groupby(['ProductName'])['Quantity'].sum().sort_values(ascending=False).fillna(0)
step5=pd.DataFrame(step5)
step5

,Quantity
ProductName,
"Salsify, Organic",4
Wine - Charddonnay Errazuriz,3
Wine - Ej Gallo Sierra Valley,3
"Oranges - Navel, 72",3
Quiche Assorted,3
...,...
"Pepsi - Diet, 355 Ml",1
"Pepper - White, Ground",1
"Peas - Pigeon, Dry",1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [11]:
a=table[[33,3909,264,3531,2503,3305]][table[33]==0]
a['Total']=a.sum(axis=1)
a.sort_values(by=['Total'],ascending=False).head(5)

CustomerID,33,3909,264,3531,2503,3305,Total
ProductName,,,,,,,
Pecan Raisin - Tarts,0.0,0.0,1.0,1.0,0.0,1.0,3.0
Eggplant - Asian,0.0,1.0,0.0,1.0,0.0,0.0,2.0
Cheese Cloth No 100,0.0,1.0,1.0,0.0,0.0,0.0,2.0
Wine - White Cab Sauv.on,0.0,0.0,0.0,1.0,0.0,1.0,2.0
Sauerkraut,0.0,0.0,1.0,1.0,0.0,0.0,2.0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [16]:
list_dic=dict()
customer_id=list(data['CustomerID'].unique())
for x in customer_id:

    similarities = distances[x].sort_values(ascending=False)[1:6]
    step5=step1.loc[similarities.keys()]
    step5=step5.groupby(['ProductName'])['Quantity'].sum().sort_values(ascending=False).fillna(0)
    step5=pd.DataFrame(step5)
    s7=step5.merge(table,how='left',on='ProductName')
    s7=s7[s7[x]==0].head(5).reset_index()
    list_dic[x]=list(s7['ProductName'])


In [18]:
list_dic

{61288: ['Mushrooms - Black, Dried',
  'Wine - Magnotta, Merlot Sr Vqa',
  'Chicken - Soup Base',
  'Wine - Chardonnay South',
  'Milk - 1%'],
 77352: ['Guinea Fowl',
  'Grenadine',
  'Oranges - Navel, 72',
  'Ecolab - Mikroklene 4/4 L',
  'Shrimp - Baby, Warm Water'],
 40094: ['Water - Mineral, Natural',
  'Oregano - Dry, Rubbed',
  'Pasta - Orecchiette',
  'Quiche Assorted',
  'Tuna - Salad Premix'],
 23548: ['Wanton Wrap',
  'Banana Turning',
  'Flavouring - Orange',
  'Chocolate - Semi Sweet, Calets',
  'Lettuce - Treviso'],
 78981: ['Lettuce - Frisee',
  'Longos - Chicken Wings',
  'Pop Shoppe Cream Soda',
  'Beef - Inside Round',
  'Sprouts - Alfalfa'],
 83106: ['Cheese - Boursin, Garlic / Herbs',
  'Garlic - Peeled',
  'Soup - Campbells Tomato Ravioli',
  'Beef - Rib Eye Aaa',
  'Wine - Chablis 2003 Champs'],
 11253: ['Smirnoff Green Apple Twist',
  'Lettuce - Treviso',
  'Tomatoes Tear Drop',
  'Juice - Cranberry, 341 Ml',
  'Cumin - Whole'],
 35107: ['Flavouring - Orange',
  '

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [20]:
s8=pd.DataFrame.from_dict(list_dic,orient='index')
s8

,0,1,2,3,4
61288,"Mushrooms - Black, Dried","Wine - Magnotta, Merlot Sr Vqa",Chicken - Soup Base,Wine - Chardonnay South,Milk - 1%
77352,Guinea Fowl,Grenadine,"Oranges - Navel, 72",Ecolab - Mikroklene 4/4 L,"Shrimp - Baby, Warm Water"
40094,"Water - Mineral, Natural","Oregano - Dry, Rubbed",Pasta - Orecchiette,Quiche Assorted,Tuna - Salad Premix
23548,Wanton Wrap,Banana Turning,Flavouring - Orange,"Chocolate - Semi Sweet, Calets",Lettuce - Treviso
78981,Lettuce - Frisee,Longos - Chicken Wings,Pop Shoppe Cream Soda,Beef - Inside Round,Sprouts - Alfalfa
...,...,...,...,...,...
49005,Olive - Spread Tapenade,"Veal - Inside, Choice",Mayonnaise - Individual Pkg,Cumin - Whole,Sardines
41286,Veal - Inside,Bread Foccacia Whole,Tea - Earl Grey,Rabbit - Whole,"Veal - Inside, Choice"
85878,"Beef - Ground, Extra Lean, Fresh",Cookie - Dough Variety,Muffin - Carrot Individual Wrap,Bread - Rye,"Mushrooms - Black, Dried"
68506,Pasta - Cheese / Spinach Bauletti,"Doilies - 5, Paper",Tea - Decaf Lipton,Black Currants,Cocoa Butter


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [22]:
distances = pd.DataFrame(1/(1 + squareform(pdist(table.T, 'correlation'))), index=table.columns, columns=table.columns)

In [23]:
list_dic=dict()
customer_id=list(data['CustomerID'].unique())
for x in customer_id:

    similarities = distances[x].sort_values(ascending=False)[1:6]
    step5=step1.loc[similarities.keys()]
    step5=step5.groupby(['ProductName'])['Quantity'].sum().sort_values(ascending=False).fillna(0)
    step5=pd.DataFrame(step5)
    s7=step5.merge(table,how='left',on='ProductName')
    s7=s7[s7[x]==0].head(5).reset_index()
    list_dic[x]=list(s7['ProductName'])



In [24]:
list_dic

{61288: ['Blueberries',
  'Wine - Magnotta - Cab Sauv',
  'Wine - Red, Harrow Estates, Cab',
  'Chips Potato Salt Vinegar 43g',
  'Lentils - Red, Dry'],
 77352: ['Juice - Apple Cider',
  'Grenadine',
  'Garbage Bags - Clear',
  'Mangoes',
  'Crab - Imitation Flakes'],
 40094: ['Bread - Roll, Soft White Round',
  'Mayonnaise - Individual Pkg',
  'Beef - Inside Round',
  'Truffle Cups - Brown',
  'Juice - V8 Splash'],
 23548: ['Zucchini - Yellow',
  'Wine - Crozes Hermitage E.',
  'Wine - Ruffino Chianti',
  'Olives - Stuffed',
  'Crackers Cheez It'],
 78981: ['Mussels - Frozen',
  'Onion Powder',
  'Bread - Raisin Walnut Oval',
  'Sauerkraut',
  'Garbage Bags - Clear'],
 83106: ['Pasta - Angel Hair',
  'Bread - Italian Roll With Herbs',
  'Pastry - Cheese Baked Scones',
  'Beef Wellington',
  'Pastry - Choclate Baked'],
 11253: ['Nut - Chestnuts, Whole',
  'Beef - Tenderlion, Center Cut',
  'Soup - Canadian Pea, Dry Mix',
  'Sauce - Demi Glace',
  'Wine - Gato Negro Cabernet'],
 35107: 

In [25]:
s8=pd.DataFrame.from_dict(list_dic,orient='index')
s8

,0,1,2,3,4
61288,Blueberries,Wine - Magnotta - Cab Sauv,"Wine - Red, Harrow Estates, Cab",Chips Potato Salt Vinegar 43g,"Lentils - Red, Dry"
77352,Juice - Apple Cider,Grenadine,Garbage Bags - Clear,Mangoes,Crab - Imitation Flakes
40094,"Bread - Roll, Soft White Round",Mayonnaise - Individual Pkg,Beef - Inside Round,Truffle Cups - Brown,Juice - V8 Splash
23548,Zucchini - Yellow,Wine - Crozes Hermitage E.,Wine - Ruffino Chianti,Olives - Stuffed,Crackers Cheez It
78981,Mussels - Frozen,Onion Powder,Bread - Raisin Walnut Oval,Sauerkraut,Garbage Bags - Clear
...,...,...,...,...,...
49005,Squid - Tubes / Tenticles 10/20,"Pepper - White, Ground",Cheese - Bocconcini,Crush - Cream Soda,Milk - 1%
41286,"Pork - Bacon, Double Smoked",Cinnamon Buns Sticky,"Seedlings - Mix, Organic",Cheese - Victor Et Berthold,Banana - Leaves
85878,Table Cloth 81x81 White,Apricots - Dried,Cheese - Camembert,Water - Green Tea Refresher,Halibut - Steaks
68506,"Oranges - Navel, 72",Hot Chocolate - Individual,Beef - Ground Medium,Spinach - Baby,Muffin Chocolate Individual Wrap
